In [5]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

from pof.failure_mode import FailureMode

In [6]:
html.Div(
    children=[
            dcc.Input(
            id='p_effective',
            type='number',
            value=50
        )
    ]
)

p_effective_input = dbc.FormGroup(
    [
        dbc.Label("Probability Effective", html_for="p_effective", width=2),
        dbc.Col(
            dbc.Input(
                type="number", id="p_effective", placeholder="p_effective"
            ),
            width=10,
        ),
    ],
    row=True,
)

In [7]:
import dash_bootstrap_components as dbc

email_input = dbc.FormGroup(
    [
        dbc.Label("Email", html_for="example-email-row", width=2),
        dbc.Col(
            dbc.Input(
                type="email", id="example-email-row", placeholder="Enter email"
            ),
            width=10,
        ),
    ],
    row=True,
)

password_input = dbc.FormGroup(
    [
        dbc.Label("Password", html_for="example-password-row", width=2),
        dbc.Col(
            dbc.Input(
                type="password",
                id="example-password-row",
                placeholder="Enter password",
            ),
            width=10,
        ),
    ],
    row=True,
)

radios_input = dbc.FormGroup(
    [
        dbc.Label("Radios", html_for="example-radios-row", width=2),
        dbc.Col(
            dbc.RadioItems(
                id="example-radios-row",
                options=[
                    {"label": "First radio", "value": 1},
                    {"label": "Second radio", "value": 2},
                    {
                        "label": "Third disabled radio",
                        "value": 3,
                        "disabled": True,
                    },
                ],
            ),
            width=10,
        ),
    ],
    row=True,
)

form = dbc.Form([email_input, password_input, radios_input])

In [8]:
# Build App
app = JupyterDash(__name__)

# Layout
app.layout = html.Div([
    form,
])

#Execute
app.run_server(mode='inline')

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [ ]:
# Task

    # Probability Effective

        # Cost
        # Consequence

        # Trigger

            # Time

            # State (n)
                # state_name
                # state (True/False)

            # Conditions (n)
                # condition_name
                # lower_threshold
                # upper_threshold

        # Impacts

            # Time

            # States (n)
                # state_name
                # state (True / False)

            # Conditions (n)
            
                # condition_name
                # method
                # axis
                # reduction_factor / target